In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import sys
import h5py
import numpy as np 
from src.parameters import Parameters as p
from src.peaksandsigmas import Peaks_and_Sigmas
from src.main import Main
from src.avgspectrum import Avg_Spectrum
def File(Input):
    # reading h5 file
    f = h5py.File(Input)
    photE = f.get("x-axis")
    intense = f.get("y-axis")
    spectrum = range(0,len(intense[1,:]-1)) # for parameter redefinition
    p.x = np.asarray(spectrum)
    
    fn = Main(intense,photE)
    def analysis(Input1):
        number_of_peaks = fn.number_of_peaks
        avgsigma = fn.avgsigma
        print("Maximum number of peaks for single spectrum in this dataset:", int(max(number_of_peaks)))
        characters = []
        indices = []
        for i in range(len(Input1)):
            if Input1[i] == ',':
                pass
            elif Input1[i] == ' ':
                pass
            else:
                integer = int(Input1[i])
                characters = np.append(characters, integer)
        characters = characters.astype(int)
        return Peaks_and_Sigmas(number_of_peaks, avgsigma, photE, intense, characters)


    button = widgets.Button(description="Plot avg spectrum")
    output = widgets.Output()

    display(button, output)

    def on_button_clicked(b):
        with output:
            return Avg_Spectrum(intense, photE)

    button.on_click(on_button_clicked)

    print("Input the spectra for a given number of peaks you wish to see, separated by commas")
    interact(analysis, Input1 = '1')

interact(File, Input = 'File name')

interactive(children=(Text(value='File name', description='Input'), Output()), _dom_classes=('widget-interact'…

<function __main__.File(Input)>

In [ ]:
# 3_stage_phase_3 & 4 - check first
#avg spectra 
# 2 stage linac one : narrower spike width